<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Fake-news-detection" data-toc-modified-id="Fake-news-detection-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Fake news detection</a></span><ul class="toc-item"><li><span><a href="#Feature-extraction-functions" data-toc-modified-id="Feature-extraction-functions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Feature extraction functions</a></span></li><li><span><a href="#Reading-and-preparing-the-corpus" data-toc-modified-id="Reading-and-preparing-the-corpus-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Reading and preparing the corpus</a></span></li><li><span><a href="#Parametrization-and-feature-extraction" data-toc-modified-id="Parametrization-and-feature-extraction-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Parametrization and feature extraction</a></span><ul class="toc-item"><li><span><a href="#Generating-experiment-sets" data-toc-modified-id="Generating-experiment-sets-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Generating experiment sets</a></span></li></ul></li><li><span><a href="#Frequency-threshold" data-toc-modified-id="Frequency-threshold-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Frequency threshold</a></span></li><li><span><a href="#Weighting-schemes" data-toc-modified-id="Weighting-schemes-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Weighting schemes</a></span></li><li><span><a href="#Classification-Process---Training" data-toc-modified-id="Classification-Process---Training-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Classification Process - Training</a></span><ul class="toc-item"><li><span><a href="#Initializing-classification-algorithms" data-toc-modified-id="Initializing-classification-algorithms-1.6.1"><span class="toc-item-num">1.6.1&nbsp;&nbsp;</span>Initializing classification algorithms</a></span></li></ul></li><li><span><a href="#End-to-End-Pipeline" data-toc-modified-id="End-to-End-Pipeline-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>End-to-End Pipeline</a></span></li><li><span><a href="#Classification-Process---Testing" data-toc-modified-id="Classification-Process---Testing-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Classification Process - Testing</a></span></li></ul></li></ul></div>

# Fake news detection

In [ ]:
# needed to make it run on Alisa's laptop 
import sys 
#sys.executable, sys.path
sys.path.append('/Users/alisa/workspace/Urdu_Fake_News/envname/lib/python3.7/site-packages')

In [1]:
import re
import glob
import numpy as np
import os
import json
import argparse
import time
import codecs
import string
import codecs
import random
import scipy.sparse as sp

from random import randrange
from scipy.sparse import csr_matrix, csc_matrix, hstack, coo_matrix
from gensim.matutils import Scipy2Corpus, corpus2csc
from gensim.models.logentropy_model import LogEntropyModel
from collections import defaultdict, Counter
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, make_scorer, accuracy_score
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords as sw
from string import punctuation

## Feature extraction functions

In [2]:
#Extracts word-ngrams, when n=1 is equal to bag of words
def wordNgrams(text, n):
    ngrams = []
    text = [word for word in text.split() if word not in string.punctuation]
    ngrams = [' '.join(text[i:i+n])+'' for i in range(len(text)-n+1)]
    return ngrams

In [ ]:
text='پاکستان کے وزیراعظم عمران خان سعودی عرب کے دارالحکومت ریاض میں ملک میں سرمایہ کاری کے حوالے سے سالانہ کانفرنس میں شرکت کر رہے ہیں حکومت پاکستان کا کہنا ہے کہ سعودی عرب نے پاکستان کو معاشی بحران سے نمٹنے میں مدد کے لیے ایک سال کے لیے تین ارب ڈالر دینے پر اتفاق کیا ہے دفترِ خارجہ کی جانب سے منگل کی شب جاری ہونے والے اعلامیے میں بتایا گیا ہے کہ یہ فیصلہ'
wordNgrams(text,2)

In [3]:
#Extracts character n-grams
def charNgrams(text, n):
    ngrams = []
    ngrams = [text[i:i+n]+'_cng' for i in range(len(text)-n+1)]
    return ngrams

In [ ]:
charNgrams(text, 1)

In [4]:
def load_diccionario(ruta):
    terms = set()#Dictionary of slangs
    try:
        tmp = open(ruta, "r")     
        while True :
            linea = tmp.readline()                                                                                   
            #linea = to_unicode(linea) 
            if (not linea) or (linea == ""):                                                                               
                break;                                                                                                      
            linea = linea.rstrip()
            terms.add(linea.lower())
        return (terms)
    except IOError as e:
        print ("Error: "+ruta+" I/O error({0}): {1}".format(e.errno, e.strerror))
        exit(1)

In [5]:
#Extracts function words n-grams with a pre-loaded dictionary
def funcNgrams(text, n):
    stop_words = load_diccionario('stop_words.txt')
    patt=r'\b(' + ('|'.join(re.escape(key) for key in stop_words)).lstrip('|') + r')\b'
    pattern = re.compile(patt)
    text = re.sub(r"(\n+|\r+|(\r\n)+)", " ", text)
    text = re.sub(r" +", " ", text)
    text = re.sub(r"’", "'", text)
    text = re.sub(r"[" + punctuation + "]*", "", text)
    terms = pattern.findall(text)
    n_grams=[('_'.join(terms[i:i+n])) + "_fwn" for i in range(len(terms)-n+1)]

    return n_grams

In [ ]:
len(funcNgrams(text, 2))

In [6]:
def extract_features(text,cn,wn,fn):
    text = text.lower()
    #text=clean_text(text)
    features = []
    for n in wn:
        if n != 0:
            features.extend(wordNgrams(text,n))
    for n in cn:
        if n != 0:
            features.extend(charNgrams(text,n))
    for n in fn:
            if n != 0:
                features.extend(funcNgrams(text,n))
    return features

In [7]:
# Extracts all features in a set of 'texts' and return as a string separated with the simbol '&%$'
def process_texts(texts,cn,wn,fn):
    occurrences=defaultdict(int)
    featuresList=[]
    featuresDict=Counter()
    for (text) in texts:
        features=extract_features(text,cn,wn,fn)
        featuresDict.update(features)
        featuresList.append('&%$'.join(features))
    return featuresList, featuresDict

## Reading and preparing the corpus

In [8]:
def preprocessText(text):
    #here remove text
    cleantext=re.sub("\d+", "0", text)
    return cleantext

In [9]:
#utility function for reading files
def read_txt_files(files):
    text=[]
    topic=[]
    for i,file_path in enumerate(files):
        print('news',file_path)
        with open(file_path,'r') as infile:
            cleantext=preprocessText(infile.read())
            text.append(cleantext)
            #print(file_path)
            file_topic=''.join(re.findall('[A-Za-z]',file_path.split('/')[3].split('.')[0]))
            #print (file_topic)
            topic.append(file_topic)
    return text, topic

In [10]:
#reading the path of real and fake news for training
train_path_real='Corpus/Train/Real/'
train_path_fake='Corpus/Train/Fake/'

real_news, real_news_topics = read_txt_files(sorted(glob.glob(train_path_real+'*.txt')))
fake_news, fake_news_topics = read_txt_files(sorted(glob.glob(train_path_fake+'*.txt')))


news Corpus/Train/Real/bus1.txt
news Corpus/Train/Real/bus10.txt
news Corpus/Train/Real/bus11.txt
news Corpus/Train/Real/bus12.txt
news Corpus/Train/Real/bus13.txt
news Corpus/Train/Real/bus14.txt
news Corpus/Train/Real/bus15.txt
news Corpus/Train/Real/bus16.txt
news Corpus/Train/Real/bus17.txt
news Corpus/Train/Real/bus18.txt
news Corpus/Train/Real/bus19.txt
news Corpus/Train/Real/bus2.txt
news Corpus/Train/Real/bus20.txt
news Corpus/Train/Real/bus21.txt
news Corpus/Train/Real/bus22.txt
news Corpus/Train/Real/bus23.txt
news Corpus/Train/Real/bus24.txt
news Corpus/Train/Real/bus25.txt
news Corpus/Train/Real/bus26.txt
news Corpus/Train/Real/bus27.txt
news Corpus/Train/Real/bus28.txt
news Corpus/Train/Real/bus29.txt
news Corpus/Train/Real/bus3.txt
news Corpus/Train/Real/bus30.txt
news Corpus/Train/Real/bus31.txt
news Corpus/Train/Real/bus32.txt
news Corpus/Train/Real/bus33.txt
news Corpus/Train/Real/bus34.txt
news Corpus/Train/Real/bus35.txt
news Corpus/Train/Real/bus36.txt
news Corpus/T

news Corpus/Train/Real/tch17.txt
news Corpus/Train/Real/tch18.txt
news Corpus/Train/Real/tch19.txt
news Corpus/Train/Real/tch2.txt
news Corpus/Train/Real/tch20.txt
news Corpus/Train/Real/tch21.txt
news Corpus/Train/Real/tch22.txt
news Corpus/Train/Real/tch23.txt
news Corpus/Train/Real/tch24.txt
news Corpus/Train/Real/tch25.txt
news Corpus/Train/Real/tch26.txt
news Corpus/Train/Real/tch27.txt
news Corpus/Train/Real/tch28.txt
news Corpus/Train/Real/tch29.txt
news Corpus/Train/Real/tch3.txt
news Corpus/Train/Real/tch30.txt
news Corpus/Train/Real/tch31.txt
news Corpus/Train/Real/tch32.txt
news Corpus/Train/Real/tch33.txt
news Corpus/Train/Real/tch34.txt
news Corpus/Train/Real/tch35.txt
news Corpus/Train/Real/tch36.txt
news Corpus/Train/Real/tch37.txt
news Corpus/Train/Real/tch38.txt
news Corpus/Train/Real/tch39.txt
news Corpus/Train/Real/tch4.txt
news Corpus/Train/Real/tch40.txt
news Corpus/Train/Real/tch41.txt
news Corpus/Train/Real/tch42.txt
news Corpus/Train/Real/tch43.txt
news Corpus/T

news Corpus/Train/Fake/tch26.txt
news Corpus/Train/Fake/tch27.txt
news Corpus/Train/Fake/tch28.txt
news Corpus/Train/Fake/tch29.txt
news Corpus/Train/Fake/tch3.txt
news Corpus/Train/Fake/tch30.txt
news Corpus/Train/Fake/tch31.txt
news Corpus/Train/Fake/tch32.txt
news Corpus/Train/Fake/tch33.txt
news Corpus/Train/Fake/tch34.txt
news Corpus/Train/Fake/tch35.txt
news Corpus/Train/Fake/tch36.txt
news Corpus/Train/Fake/tch37.txt
news Corpus/Train/Fake/tch38.txt
news Corpus/Train/Fake/tch39.txt
news Corpus/Train/Fake/tch4.txt
news Corpus/Train/Fake/tch40.txt
news Corpus/Train/Fake/tch41.txt
news Corpus/Train/Fake/tch42.txt
news Corpus/Train/Fake/tch43.txt
news Corpus/Train/Fake/tch44.txt
news Corpus/Train/Fake/tch45.txt
news Corpus/Train/Fake/tch46.txt
news Corpus/Train/Fake/tch47.txt
news Corpus/Train/Fake/tch48.txt
news Corpus/Train/Fake/tch49.txt
news Corpus/Train/Fake/tch5.txt
news Corpus/Train/Fake/tch50.txt
news Corpus/Train/Fake/tch51.txt
news Corpus/Train/Fake/tch52.txt
news Corpus/T

In [11]:
#contatenating real and fake news in one variable for training
train_texts = np.concatenate((real_news, fake_news))

train_labels_real = np.concatenate((np.ones(len(real_news)), np.zeros(len(fake_news))))
# swapped real and fake 
train_labels_fake = np.concatenate((np.zeros(len(real_news)), np.ones(len(fake_news))))

train_topics = np.concatenate((real_news_topics, fake_news_topics))

In [ ]:
print ('Train:')
print ('\t Real:',len(real_news))
print ('\t Fake:',len(fake_news))

In [12]:
#reading the path of real and fake news for testing
test_path_real='Corpus/Test/Real/'
test_path_fake='Corpus/Test/Fake/'

real_news, real_news_topics = read_txt_files(sorted(glob.glob(test_path_real+'*.txt')))
fake_news, fake_news_topics = read_txt_files(sorted(glob.glob(test_path_fake+'*.txt')))

#contatenating real and fake news in one variable for testing
test_texts = np.concatenate((real_news, fake_news))
test_labels_real = np.concatenate((np.ones(len(real_news)), np.zeros(len(fake_news))))
test_labels_fake = np.concatenate((np.zeros(len(real_news)), np.ones(len(fake_news))))
test_topics = np.concatenate((real_news_topics, fake_news_topics))

news Corpus/Test/Real/bus100.txt
news Corpus/Test/Real/bus71.txt
news Corpus/Test/Real/bus72.txt
news Corpus/Test/Real/bus73.txt
news Corpus/Test/Real/bus74.txt
news Corpus/Test/Real/bus75.txt
news Corpus/Test/Real/bus76.txt
news Corpus/Test/Real/bus77.txt
news Corpus/Test/Real/bus78.txt
news Corpus/Test/Real/bus79.txt
news Corpus/Test/Real/bus80.txt
news Corpus/Test/Real/bus81.txt
news Corpus/Test/Real/bus82.txt
news Corpus/Test/Real/bus83.txt
news Corpus/Test/Real/bus84.txt
news Corpus/Test/Real/bus85.txt
news Corpus/Test/Real/bus86.txt
news Corpus/Test/Real/bus87.txt
news Corpus/Test/Real/bus88.txt
news Corpus/Test/Real/bus89.txt
news Corpus/Test/Real/bus90.txt
news Corpus/Test/Real/bus91.txt
news Corpus/Test/Real/bus92.txt
news Corpus/Test/Real/bus93.txt
news Corpus/Test/Real/bus94.txt
news Corpus/Test/Real/bus95.txt
news Corpus/Test/Real/bus96.txt
news Corpus/Test/Real/bus97.txt
news Corpus/Test/Real/bus98.txt
news Corpus/Test/Real/bus99.txt
news Corpus/Test/Real/hlth100.txt
news 

In [ ]:
print ('Test:')
print ('\t Real:',len(real_news))
print ('\t Fake:',len(fake_news))

In [45]:
train_labels_dict = {'real': train_labels_real, 'fake': train_labels_fake}
test_labels_dict = {'real': test_labels_real, 'fake': test_labels_fake}

## Parametrization and feature extraction

### Generating experiment sets 

We're generating a few sets of experiments with different varyingvalues of N-grams for each feature type and their combinations.  This looks like incomplete exhaustive search because we actually don't consider (at least so far) _all_ possible combinations that sрould be roughly 3^6 = 729 and then multiply by varios normalization techniques by various classifiers. 

We actually can do it yet I am not sure whether it'll make sense and maybe there's a better "smart grid search"  strategy when we first lok at some parameter combination, then find the best and then look at other possible variations around that point of parameter values.  

In [13]:
# [cnvalues, wnvalues, fnvalues]
num_feat_types = 3 
ngram_feat_combinations = [[], [], []]

# for a single type n-grams
for j in range(num_feat_types): 
    for i in range(1,7):
        ngram_feat_combinations[j].append([i])
        for k in range(num_feat_types): 
            if k != j: 
                ngram_feat_combinations[k].append([0])
        
# extending with 1-1-1, 2-2-2,3-3-3,4-4-4,5-5-5,6-6-6 variants
ngrams = [[x] for x in range(1,7)]

for ngram_type in ngram_feat_combinations: 
    ngram_type.extend(ngrams)
    
# combinations of varying X-grams - uni-Y
for l in range(num_feat_types):
    for j in range(num_feat_types):
        if j != l: 
            for i in range(1,7):
                ngram_feat_combinations[l].append([i])
                ngram_feat_combinations[j].append([1])    
                for k in range(num_feat_types):                 
                    if k != j and k != l: 
                        ngram_feat_combinations[k].append([0])       
        
len(ngram_feat_combinations[0])                        

60

In [ ]:
# generated experiments:  
for i in range(10):     
        for feat in ngram_feat_combinations:
                print(feat[i*6 : i*6+6])
        print()        

In [25]:
# Parameters
cnvalues=[4] #character n-grams
wnvalues=[0] # word n-grams
fnvalues=[0] # function words n-grams

In [68]:
class FeatureExtractor: 
    """Feature extraction"""
    
    def __init__(self, cnvalues, wnvalues, fnvalues): 
        print('Feature extractor for ' + str(cnvalues[0]) +'-' + str(wnvalues[0]) + '-' + str(fnvalues[0]) + ' char-word-func N-gram combination')
        self.cnvalues = cnvalues
        self.wnvalues = wnvalues
        self.fnvalues = fnvalues
        
        self.vectorizer = CountVectorizer(lowercase=False, min_df=2, tokenizer=lambda x: x.split('&%$')) #--> we can change this
        
    def fit_extract(self, train_texts):
        train_features, dicOfFeatures = process_texts(train_texts, self.cnvalues, self.wnvalues, self.fnvalues)
        train_data = self.vectorizer.fit_transform(train_features)
        train_data = train_data.astype(float)
        return train_data, dicOfFeatures
    
    def extract(self, test_texts): 
        test_features, dicOfFeaturesTest = process_texts(test_texts, self.cnvalues, self.wnvalues, self.fnvalues)
        test_data = self.vectorizer.transform(test_features)
        test_data = test_data.astype(float)
        return test_data, dicOfFeaturesTest            

In [14]:
def feature_extraction(texts, cnvalues, wnvalues, fnvalues):
    """Train feature extraction"""
    print('Extracting features for ' + str(cnvalues[0]) +'-' + str(wnvalues[0]) + '-' + str(fnvalues[0]) + ' char-word-func N-gram combination')
    features, dicOfFeatures = process_texts(texts, cnvalues, wnvalues, fnvalues)

    vectorizer = CountVectorizer(lowercase=False, min_df=2, tokenizer=lambda x: x.split('&%$')) #--> we can change this
    feature_mx = vectorizer.fit_transform(features)
    feature_mx = feature_mx.astype(float)
    return feature_mx, dicOfFeatures    

In [ ]:
train_texts

In [27]:
#Train feature extraction
print('Extracting features')
train_features, dicOfFeatures = process_texts(train_texts,cnvalues,wnvalues,fnvalues)

vectorizer = CountVectorizer(lowercase=False, min_df=2, tokenizer=lambda x: x.split('&%$')) #--> we can change this
train_data = vectorizer.fit_transform(train_features)
train_data = train_data.astype(float)
#print('\t', 'Labels for each document: ', len(train_labels))
print('\t', 'Total training files (Real + Fake) : ', len(train_texts))
print('\t', 'Vocabulary size of', len(train_texts), 'files is : ',len(dicOfFeatures))
print ('\t','Train shape:',train_data.shape)
#print('\t', 'class dictribution',Counter(train_labels))

Extracting features
	 Total training files (Real + Fake) :  638
	 Vocabulary size of 638 files is :  55261
	 Train shape: (638, 36272)


In [30]:
train_data.shape(axis=1)

TypeError: 'tuple' object is not callable

In [ ]:
# Test feature extraction
print('Extracting Test features')
test_features,dicOfFeaturesTest = process_texts(test_texts,cnvalues,wnvalues,fnvalues)
test_data = vectorizer.transform(test_features)
test_data = test_data.astype(float)

print('\t', 'texts', len(test_texts))
print('\t', 'vocabulary size',len(dicOfFeaturesTest))

print ('\t','Test shape:',test_data.shape)

## Frequency threshold

In [15]:
def apply_frequency_threshold(feature_mx, N=5):
    values=np.array(feature_mx.sum(axis=0)).ravel()
    thresholdMask=(values >= N)*1
    indices_zero = list(np.nonzero(thresholdMask == 0)[0])
    all_cols = np.arange(feature_mx.shape[1])
    cols_to_keep = np.where(np.logical_not(np.in1d(all_cols, indices_zero)))[0]
    return cols_to_keep 

#cols_to_keep = apply_frequency_threshold(feature_mx)
#thresholded_train_feat_mx = train_feature_mx[:, cols_to_keep]
#thresholded_test_feat_mx = test_feature_mx[:, cols_to_keep]

In [ ]:
N=5
X=train_data
values=np.array(X.sum(axis=0)).ravel()
thresholdMask=(values >= N)*1
indices_zero = list(np.nonzero(thresholdMask == 0)[0])
all_cols = np.arange(X.shape[1])
cols_to_keep = np.where(np.logical_not(np.in1d(all_cols, indices_zero)))[0]
train_data = X[:, cols_to_keep]
#####
scaled_train_data=train_data
print('Train shape:',scaled_train_data.shape)

In [ ]:
# umbral de frecuencia, la N ya no se define
Z=test_data
all_cols = np.arange(Z.shape[1])
cols_to_keep = np.where(np.logical_not(np.in1d(all_cols, indices_zero)))[0]
test_data = Z[:, cols_to_keep]
scaled_test_data=test_data
print('Test shape:',scaled_test_data.shape)

## Weighting schemes

In [16]:
# possible values: binary, logent, tfidf, norm, relat
weighting_schemes = ['binary', 'logent', 'tfidf', 'norm', 'relat', 'none (tf)']

In [31]:
from gensim.matutils import Scipy2Corpus, corpus2csc
from gensim.models.logentropy_model import LogEntropyModel

class LogEntWeightScheme: 
        
    def __init__(self): 
        self.log_ent = LogEntropyModel()         
        
    def fit(self, scaled_train_data):
        Xc = Scipy2Corpus(scaled_train_data)
        self.log_ent = LogEntropyModel(Xc)
        X = self.log_ent[Xc]
        X = corpus2csc(X)
        scaled_train_data = sp.csc_matrix.transpose(X)
        return scaled_train_data
        
    def fit_transform(self, scaled_train_data):    
        Xc = Scipy2Corpus(scaled_train_data)
        self.log_ent = LogEntropyModel(Xc)
        X = self.log_ent[Xc]
        X = corpus2csc(X)
        scaled_train_data = sp.csc_matrix.transpose(X)
        return scaled_train_data 

    def transform(self, scaled_test_data):
        Xtest = Scipy2Corpus(scaled_test_data)
        X = self.log_ent[Xtest]
        X = corpus2csc(X, scaled_train_data.shape[1])
        scaled_test_data = sp.csc_matrix.transpose(X)
        return scaled_test_data
        

In [32]:
from scipy.sparse import coo_matrix

class RelativeWeightScheme:
    
    def __init__(self):
        self.s = 1.0 
             
    def fit(self, scaled_train_data):
        self.s = float(scaled_train_data.sum(axis = 1))
        scaled_train_data = coo_matrix(np.nan_to_num(scaled_train_data/self.s))
        return scaled_train_data

    # identical to fit()
    def fit_transform(self, scaled_train_data):
        self.s = float(scaled_train_data.sum(axis = 1))
        scaled_train_data = coo_matrix(np.nan_to_num(scaled_train_data/self.s))
        return scaled_train_data
                
    def transform(self, scaled_test_data):                      
        scaled_test_data = coo_matrix(np.nan_to_num(scaled_test_data/self.s))
        return scaled_test_data
    

In [33]:
class DoNothingWeightScheme:
    """Making it easier to comply with APIs for a do-nothing trasnformer"""

    def __init__(self):
        pass
    
    def fit(self, scaled_train_data):
        return scaled_train_data

    def fit_transform(self, scaled_train_data):
        return scaled_train_data

    def transform(self, scaled_test_data):
        return scaled_test_data
       

In [34]:
def get_weighting_scheme(feature_weight_name): 
    
    if feature_weight_name == 'binary':
        print ("feature_weight = binary")
        return preprocessing.Binarizer()

    elif feature_weight_name == 'logent':
        print ("feature_weight = logent")
        return LogEntWeightScheme()
        
    elif feature_weight_name == 'tfidf':
        print ("feature_weight = tfidf")
        return TfidfTransformer()

    elif feature_weight_name == 'norm':
        # cannot normalize with l2 in a Pipeline
        print ("feature_weight = norm")
        return preprocessing.MaxAbsScaler()

    elif feature_weight_name == 'relat':
        print ("feature_weight = relat")
        return RelativeWeightScheme()

    else:
        print ("feature_weight = tf")
        return DoNothingWeightScheme()         

In [17]:
def apply_weighing_scheme(scaled_train_data, test_data, feature_weight): 
    print("Applying weighting scheme")

    if feature_weight == 'binary':
        scaled_train_data = preprocessing.Binarizer().transform(scaled_train_data)
        scaled_test_data = preprocessing.Binarizer().transform(scaled_test_data)
        print ("feature_weight = binary")

    elif feature_weight == 'logent':
        Xc = Scipy2Corpus(scaled_train_data)
        log_ent = LogEntropyModel(Xc)
        X = log_ent[Xc]
        X = corpus2csc(X)
        scaled_train_data = sp.csc_matrix.transpose(X)

        Xtest = Scipy2Corpus(scaled_test_data)
        X = log_ent[Xtest]
        X = corpus2csc(X, scaled_train_data.shape[1])
        scaled_test_data = sp.csc_matrix.transpose(X)
        print ("feature_weight = logent")

    elif feature_weight == 'tfidf':
        transformer = TfidfTransformer()
        scaled_train_data = transformer.fit_transform(scaled_train_data)
        scaled_test_data = transformer.transform(scaled_test_data)
        print ("feature_weight = tfidf")

    elif feature_weight=='norm':
        #scaled_train_data = preprocessing.normalize(scaled_train_data, norm='l2')
        #Scaling data
        scaled_train_data = preprocessing.normalize(scaled_train_data, norm='l2')
        max_abs_scaler = preprocessing.MaxAbsScaler()
        scaled_train_data = max_abs_scaler.fit_transform(scaled_train_data)
        scaled_test_data = max_abs_scaler.transform(scaled_test_data)
        print ("feature_weight = norm")

    elif feature_weight=='relat':
        s = scaled_train_data.sum(axis = 1)
        scaled_train_data = coo_matrix(np.nan_to_num(scaled_train_data/s))

        #s = scaled_test_data.sum(axis = 1)
        scaled_test_data = coo_matrix(np.nan_to_num(scaled_test_data/s))
        print ("feature_weight = relat")

    else:
        print ("feature_weight = tf")

    return scaled_train_data, scaled_test_data       

In [ ]:
#print ('only frecuency:',test_data)
feature_weight='logent' # possible values: binary, logent, tfidf, norm, relat
print ('Train:',scaled_train_data.shape)
print ('Test:',scaled_test_data.shape)

if feature_weight == 'binary':
    scaled_train_data = preprocessing.Binarizer().fit_transform(scaled_train_data)
    scaled_test_data = preprocessing.Binarizer().fit_transform(scaled_test_data)
    print ("feature_weight = binary")
    
elif feature_weight == 'logent':
    Xc = Scipy2Corpus(scaled_train_data)
    log_ent = LogEntropyModel(Xc)
    X = log_ent[Xc]
    X = corpus2csc(X)
    scaled_train_data = sp.csc_matrix.transpose(X)
    
    Xtest = Scipy2Corpus(scaled_test_data)
    X = log_ent[Xtest]
    X = corpus2csc(X, scaled_train_data.shape[1])
    scaled_test_data = sp.csc_matrix.transpose(X)
    print ("feature_weight = logent")
    
elif feature_weight == 'tfidf':
    transformer = TfidfTransformer()
    scaled_train_data = transformer.fit_transform(scaled_train_data)
    scaled_test_data = transformer.transform(scaled_test_data)
    print ("feature_weight = tfidf")
    
elif feature_weight=='norm':
    #scaled_train_data = preprocessing.normalize(scaled_train_data, norm='l2')
    #Scaling data
    max_abs_scaler = preprocessing.MaxAbsScaler()
    scaled_train_data = max_abs_scaler.fit_transform(scaled_train_data)
    scaled_test_data = max_abs_scaler.transform(scaled_test_data)
    print ("feature_weight = norm")
    
elif feature_weight=='relat':
    s = scaled_train_data.sum(axis = 1)
    scaled_train_data = coo_matrix(np.nan_to_num(scaled_train_data/s))


    #s = scaled_test_data.sum(axis = 1)
    scaled_test_data = coo_matrix(np.nan_to_num(scaled_test_data/s))
    print ("feature_weight = relat")
    
else:
    print ("feature_weight = tf")
    
print ('Train:',scaled_train_data.shape)
print ('Test:',scaled_test_data.shape)

## Classification Process - Training

In [79]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from IPython.display import display
from mlxtend.plotting import plot_decision_regions

def check_model_fit(clf, X_test, y_test):
    # Print overall test-set accuracy
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred, normalize=True)
    #print('total accuracy = {:.1f}%'.format(acc))

    p = precision_score(y_test, y_pred)
    r = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
       
    balanced_acc = balanced_accuracy_score(y_test, y_pred)     
    
    y_pred_scores = clf.predict_proba(X_test)
    roc_auc = roc_auc_score(y_test, y_pred_scores[:, 1])        
    
    # Print confusion matrix
    #cmat = confusion_matrix(y_test, y_pred)
    #cols = pd.MultiIndex.from_tuples([('predictions', 0), ('predictions', 1)])
    #indx = pd.MultiIndex.from_tuples([('actual', 0), ('actual', 1)])
    #display(pd.DataFrame(cmat, columns=cols, index=indx))
    #print()
    
    # Print test-set accuracy grouped by the target variable 
    #print('percent accuracy score per class:')
    #cmat = confusion_matrix(y_test, y_pred)
    #scores = cmat.diagonal() / cmat.sum(axis=1) * 100
    # specificity: 
    #irrelevant_acc = scores[0]
    # sensetivity:  
    #relevant_acc = scores[1]
    #balanced_acc = (relevant_acc + irrelevant_acc)/2.0
    #print('left = 0 : {:.2f}%'.format(scores[0]))
    #print('left = 1 : {:.2f}%'.format(scores[1]))
    #print()
    
    # Plot decision regions
    #fig = plt.figure(figsize=(8, 8))
    #N_samples = 200
    #X, y = X_test[:N_samples], y_test[:N_samples]
    #plot_decision_regions(X, y, clf=clf)
    
    #plt.xlabel('satisfaction_level')
    #plt.ylabel('last_evaluation')
    #plt.legend(loc='upper left')
    
    results = {'accuracy': acc, 
               'balanced_accuracy': balanced_acc, 
               'precision': p, 
               'recall': r, 
               'f1': f1, 
               'roc_auc': roc_auc}
    
    return results
    

In [ ]:
#Utility function
originalclass=[]
predictedclass=[]
def classification_report_with_f1_score(y_true, y_pred):
    originalclass.extend(y_true)
    predictedclass.extend(y_pred)
    return f1_score(y_true, y_pred) # return accuracy score

In [ ]:
print('Training Classifier')
    
# Applying classification algorithms
clf=LinearSVC(C=0.01,class_weight='balanced', random_state=85)
clfSVC=SVC(C=0.01, kernel='linear',class_weight='balanced')
clfMnb=MultinomialNB()
clfBnb=BernoulliNB()
clfLG=LogisticRegression(solver='lbfgs', tol=0.001, C=0.01,class_weight='balanced')

clf.fit(scaled_train_data, train_labels)
nested_score = cross_val_score(clf, X=scaled_train_data, y=train_labels, cv=10, scoring=make_scorer(classification_report_with_f1_score))
#cvScoreLinearSVC=cross_val_score(clf, scaled_train_data, train_labels, cv=10, scoring='f1').mean()
print(classification_report(originalclass, predictedclass))
print('10-Fold Cross-validation Linear SVC',nested_score.mean())

cvScoreLG=cross_val_score(clfLG, scaled_train_data, train_labels, cv=10, scoring='f1').mean()
print('10-Fold Cross-validation Logistic Regression',cvScoreLG)

cvScoreMnb=cross_val_score(clfMnb, scaled_train_data, train_labels, cv=10, scoring='f1').mean()
print('10-Fold Cross-validation Multinomial Naive Bayes',cvScoreMnb)

cvScoreBnb=cross_val_score(clfBnb, scaled_train_data, train_labels, cv=10, scoring='f1').mean()
print('10-Fold Cross-validation Bernoulli Naive Bayes',cvScoreBnb)

### Initializing classification algorithms

In [36]:
# AZ:  I don't think we need to balance the classes: 
# first, the data is not so heavily imbalanced 
# second, we loose the class distribution information  

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.ensemble.weight_boosting import AdaBoostClassifier

clfLG = LogisticRegression(solver='lbfgs', tol=0.001, C=0.01)
clfLinSVM = LinearSVC(C=0.01, random_state=85)
clfSVC = SVC(C=0.01, kernel='linear')
clfDT = DecisionTreeClassifier(random_state=0)
clfRFC = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clfMnb = MultinomialNB()
clfBnb = BernoulliNB()
clfAB = AdaBoostClassifier(base_estimator=None, n_estimators=50, learning_rate=1.0,  random_state=None)


In [84]:
clfs = [clfLG, clfLinSVM, clfSVC, clfDT, clfRFC, clfMnb, clfBnb,  clfAB]

In [72]:
from sklearn.model_selection import cross_validate

def cv(clf, train_X, train_y, scoring=['accuracy', 'balanced_accuracy' ,'precision', 'recall', 'f1', 'roc_auc']): 
    cv = cross_validate(clf, train_X, train_y, cv=5, return_estimator=True, scoring=scoring)
    
    output = {}
    for s in scoring: 
        m = np.mean(cv['test_' + s])
        v = np.var(cv['test_' + s])
        output[s] ={'mean': m, 'var': v}      
    return output

## End-to-End Pipeline

Includes running a set of experiments  and outputs the values  

* Doesn't include reading in texts 

In [49]:
import pandas as pd 

report = pd.DataFrame(columns=['N-gram-feat-comb', 'total_feat_num', 'weight scheme', 'classifier', 'cv-acc', 'cv-balanced_acc','cv-roc', 'cv-f1-fake', 'acc', 'balanced_acc', 'roc', 'f1_fake', 'f1_real'])
report

,N-gram-feat-comb,total_feat_num,weight scheme,classifier,cv-acc,cv-balanced_acc,cv-roc,cv-f1-fake,acc,balanced_acc,roc,f1_fake,f1_real


In [86]:
from sklearn.pipeline import Pipeline

def end_to_end(train_texts, train_labels_dict, test_texts, test_labels_dict, 
             ngram_feat_sets, weight_schemes, clfs, 
             expLog_df, name_prefix=''):  
    
    all_experiments = {}
    
    # for each combination of ngram feature types
    for i in range(len(ngram_feat_sets[0])):         
        cnvalues = ngram_feat_sets[0][i]
        wnvalues = ngram_feat_sets[1][i] 
        fnvalues = ngram_feat_sets[2][i] 
        ngram_feat_type_set_str = str(cnvalues[0]) +'-' + str(wnvalues[0]) + '-' + str(fnvalues[0]) + ' char-word-func'
        
        feat_extractor = FeatureExtractor(cnvalues, wnvalues, fnvalues)        
        train_feature_mx, _ = feat_extractor.fit_extract(train_texts)
        test_feature_mx, _ = feat_extractor.extract(test_texts) 
        print('train shape: ', train_feature_mx.shape, 'test shape:', test_feature_mx.shape)
        
        #applying the frequency threshold
        cols_to_keep = apply_frequency_threshold(train_feature_mx)
        thresholded_train_feat_mx = train_feature_mx[:, cols_to_keep]
        thresholded_test_feat_mx = test_feature_mx[:, cols_to_keep]
        
        #for each weighting scheme, for each classifier, create a pipeline
        pipelines = []
        
        all_experiments[ngram_feat_type_set_str] = {}
        for weight_scheme_name in weight_schemes: 
            all_experiments[ngram_feat_type_set_str][weight_scheme_name] = {}
            weight_scheme = get_weighting_scheme(weight_scheme_name)
            for clf in clfs: 
                clf_name = type(clf).__name__
                print('Classifier: ' + clf_name)
                all_experiments[ngram_feat_type_set_str][weight_scheme_name][clf_name] = {}
                ppl = Pipeline([('scaler', weight_scheme), ('clf', clf)])

                cv_scores_by_labeling = {}
                test_scores_by_labeling = {}
                for key in train_labels_dict:
                    print(key + " run...")
                    train_X = thresholded_train_feat_mx.copy()
                    train_X_cv = thresholded_train_feat_mx.copy()
                    test_X = thresholded_test_feat_mx.copy() 

                    train_y = train_labels_dict[key]
                    test_y = test_labels_dict[key]
                    
                    # cross-validation run 
                    # scoring=['accuracy', 'balanced_accuracy' ,'precision', 'recall', 'f1', 'roc_auc']
                    cv_scores = cv(ppl, train_X_cv, train_y)
                    cv_scores_by_labeling[key] = cv_scores
                    
                    # test run 
                    ppl.fit(train_X, train_y)
                    #acc, balanced_acc, p, r, f1, roc_auc
                    test_scores = check_model_fit(ppl, test_X, test_y)
                    test_scores_by_labeling[key] = test_scores
                
                all_experiments[ngram_feat_type_set_str][weight_scheme_name][clf_name]['cv'] = cv_scores_by_labeling
                all_experiments[ngram_feat_type_set_str][weight_scheme_name][clf_name]['test'] = test_scores_by_labeling
                                
                # record experiments to the log 
                #['N-gram-feat-comb', 'total_feat_num', 'weight scheme', 'classifier', 
                # 'cv-acc', 'cv-balanced_acc','cv-roc', 'cv-f1-fake', 
                # 'acc', 'balanced_acc', 'roc', 'f1_fake', 'f1_real']
                expLog_df.loc[len(expLog_df)] = [name_prefix + ngram_feat_type_set_str, thresholded_train_feat_mx.shape[1], weight_scheme_name, clf_name,
                                           np.round(cv_scores_by_labeling['fake']['accuracy']['mean'], 2), np.round(cv_scores_by_labeling['fake']['balanced_accuracy']['mean'], 2), np.round(cv_scores_by_labeling['fake']['roc_auc']['mean'], 2), np.round(cv_scores_by_labeling['fake']['f1']['mean'], 2), 
                                           np.round(test_scores_by_labeling['fake']['accuracy'], 2), np.round(test_scores_by_labeling['fake']['balanced_accuracy'], 2), np.round(test_scores_by_labeling['fake']['roc_auc'], 2), np.round(test_scores_by_labeling['fake']['f1'], 2), np.round(test_scores_by_labeling['real']['f1'], 2)]
                
                expLog_df
                
    return all_experiments, expLog_df

In [81]:
# small experiment 

my_expLog = pd.DataFrame(columns=['N-gram-feat-comb', 'total_feat_num', 'weight scheme', 'classifier', 'cv-acc', 'cv-balanced_acc','cv-roc', 'cv-f1-fake', 'acc', 'balanced_acc', 'roc', 'f1_fake', 'f1_real'])

#ngram_feat_combinations

weighting_schemes = ['binary', 'logent', 'tfidf', 'norm', 'relat', 'none (tf)']
#my_weight_schemes = ['binary']
#my_clfs = [clfLG]
clfs = [clfLG, clfLinSVM, clfSVC, clfDT, clfRFC, clfMnb, clfBnb,  clfAB]

all_results, log = end_to_end(train_texts, train_labels_dict, test_texts, test_labels_dict, 
             ngram_feat_combinations, my_weight_schemes, my_clfs, 
             my_expLog)


train shape:  (638, 110) test shape: (262, 110)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 2029) test shape: (262, 2029)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 11666) test shape: (262, 11666)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 36272) test shape: (262, 36272)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 73096) test shape: (262, 73096)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 108412) test shape: (262, 108412)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 8347) test shape: (262, 8347)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 31621) test shape: (262, 31621)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 38

/Users/z0034bv/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/z0034bv/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/z0034bv/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/z0034bv/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.

train shape:  (638, 28389) test shape: (262, 28389)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun


/Users/z0034bv/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/z0034bv/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/z0034bv/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/z0034bv/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.

train shape:  (638, 310) test shape: (262, 310)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 6968) test shape: (262, 6968)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 19001) test shape: (262, 19001)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 21822) test shape: (262, 21822)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun


...

train shape:  (638, 19345) test shape: (262, 19345)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun


/Users/z0034bv/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/z0034bv/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/z0034bv/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/z0034bv/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.

train shape:  (638, 17413) test shape: (262, 17413)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun


/Users/z0034bv/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/z0034bv/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/z0034bv/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/z0034bv/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.

train shape:  (638, 8767) test shape: (262, 8767)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 40618) test shape: (262, 40618)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 69286) test shape: (262, 69286)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 93928) test shape: (262, 93928)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 124248) test shape: (262, 124248)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 154214) test shape: (262, 154214)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 8457) test shape: (262, 8457)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (638, 10376) test shape: (262, 10376)
feature_weight = binary
Classifier: LogisticRegression
realrun
fakerun
train shape:  (6

In [82]:
log

,N-gram-feat-comb,total_feat_num,weight scheme,classifier,cv-acc,cv-balanced_acc,cv-roc,cv-f1-fake,acc,balanced_acc,roc,f1_fake,f1_real
0,my-[1]-char+[0]-word+[0]-func,101,binary,LogisticRegression,0.66,0.64,0.79,0.52,0.75,0.72,0.85,0.65,0.80
1,my-[2]-char+[0]-word+[0]-func,1555,binary,LogisticRegression,0.79,0.79,0.89,0.78,0.85,0.84,0.94,0.81,0.88
2,my-[3]-char+[0]-word+[0]-func,7927,binary,LogisticRegression,0.68,0.68,0.81,0.64,0.82,0.80,0.90,0.77,0.85
3,my-[4]-char+[0]-word+[0]-func,20681,binary,LogisticRegression,0.66,0.65,0.75,0.60,0.79,0.76,0.85,0.71,0.83
4,my-[5]-char+[0]-word+[0]-func,31926,binary,LogisticRegression,0.64,0.64,0.72,0.59,0.75,0.72,0.83,0.64,0.81
5,my-[6]-char+[0]-word+[0]-func,37105,binary,LogisticRegression,0.63,0.63,0.70,0.58,0.74,0.71,0.81,0.63,0.80
6,my-[0]-char+[1]-word+[0]-func,4020,binary,LogisticRegression,0.65,0.64,0.73,0.57,0.74,0.72,0.79,0.63,0.80
7,my-[0]-char+[2]-word+[0]-func,6690,binary,LogisticRegression,0.64,0.63,0.68,0.51,0.67,0.63,0.72,0.48,0.76
8,my-[0]-char+[3]-word+[0]-func,2860,binary,LogisticRegression,0.62,0.59,0.68,0.35,0.61,0.56,0.65,0.29,0.73
9,my-[0]-char+[4]-word+[0]-func,847,binary,LogisticRegression,0.59,0.55,0.65,0.18,0.58,0.52,0.59,0.08,0.73


In [75]:
!mkdir results

In [83]:
log.to_csv('./results/results-binary-LogReg-only.csv', index=False)

## Classification Process - Testing

In [ ]:
predictions=clf.predict(scaled_test_data)
print(classification_report(test_labels, predictions))
print('Accuracy',accuracy_score(test_labels, predictions))
print('F1-score',f1_score(test_labels, predictions))